In [1]:
import socket
import pandas as pd
import numpy as np

In [6]:
filename = './Data/5-130MHz_126pts_ALL'
data_points = 126

In [3]:
# Fonction pour convertir une chaîne hexadécimale en une liste de valeurs décimales
def hex_to_dec(hex_str):
    hex_value = '0x' + hex_str.replace(' ', '')
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    return dec_value

def hex_to_dec_little_endian_C2(hex_str):
    # Diviser la chaîne en octets
    hex_values = hex_str.split()
    # Réorganiser les octets en ordre inverse (little-endian)
    hex_values.reverse()
    # Concaténer les octets et ajouter le préfixe '0x'
    # Prendre les 2,5 derniers octets (20 bits de poids faible)
    hex_value = '0x' + hex_values[1][1] + hex_values[2] + hex_values[3]
    print(hex_value)
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    dec_value -= (1 << 20)
    return dec_value

def hex_to_dec_little_endian(hex_str):
    # Diviser la chaîne en octets
    hex_values = hex_str.split()
    # Réorganiser les octets en ordre inverse (little-endian)
    hex_values.reverse()
    # Concaténer les octets et ajouter le préfixe '0x'
    # Prendre les 2,5 derniers octets (20 bits de poids faible)
    hex_value = '0x' + hex_values[0] + hex_values[1] + hex_values[2] + hex_values[3]
    print(hex_value)
    # Convertir en décimal
    dec_value = int(hex_value, 16)
    return dec_value

def S_to_Z(S_real, S_imag, Z0=50):
    """
    Calcule l'impédance réelle et imaginaire à partir des paramètres S.

    :param S_real: Liste des parties réelles des paramètres S.
    :param S_imag: Liste des parties imaginaires des paramètres S.
    :param Z0: Impédance caractéristique (par défaut 50 ohms).
    :return: Tuple contenant les parties réelles et imaginaires de l'impédance.
    """
    # Convertir les listes en nombres complexes
    S = np.array(S_real) + 1j * np.array(S_imag)

    # Calculer l'impédance
    Z = Z0 * (1 + S) / (1 - S)

    # Extraire les parties réelles et imaginaires
    Z_real = Z.real
    Z_imag = Z.imag

    return Z_real, Z_imag

# Get data

In [4]:
def send_hex_data_and_receive(ip, port, hex_data, timeout=10):
    # Convertir la chaîne hexadécimale en bytes
    data = bytes.fromhex(hex_data)

    # Créer un socket TCP/IP
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        try:
            # Se connecter au serveur
            s.connect((ip, port))
            print(f"Connecté à {ip} sur le port {port}")

            # Envoyer les données
            s.sendall(data)
            print(f"Données envoyées: {hex_data}")

            # Définir le timeout pour la réception
            s.settimeout(timeout)

            # Recevoir plusieurs paquets de 1040 bytes
            total_data = b''
            while True:
                try:
                    response = b''
                    response = s.recv(1050)
                    if not response:
                        # Si la réponse est vide, arrêter la réception
                        break
                    total_data += response
                    print(f"Paquet reçu de {len(response)} bytes")
                    print(f"Réponse complète reçue: {response.decode('ascii', errors='ignore')}")
                    with open(filename + '.bin', 'ab') as file:
                        file.write(response)
                except socket.timeout:
                    print("Timeout atteint, fin de la réception des données.")
                    break

            # Afficher la réponse reçue en ASCII
            print(f"Réponse complète reçue: {total_data.decode('ascii', errors='ignore')}")

        except Exception as e:
            print(f"Une erreur est survenue: {e}")
        finally:
            # Fermer la connexion proprement
            try:
                s.close()
                print("Connexion fermée")
            except Exception as e:
                print(f"Une erreur est survenue lors de la fermeture de la connexion: {e}")

In [25]:
# Définir les paramètres de connexion
ip_address = '192.168.1.38'
port_number = 66
Hello = '7F 41 50 01 00 92 7e'
Get_smith = '7f 41 11 01 00 64 65 66 61 75\
6c 74 2e 74 72 61 00 8f 54 18 1d 90 54 18 1d 50\
82 16 1d 00 00 e3 1b 00 00 00 00 fc ec 55 07 46\
f1 a1 77 00 00 00 00 00 00 00 00 00 00 00 00 88\
f3 55 07 90 54 18 1d 00 00 00 00 48 ed 55 07 1b\
3c 0f 75 00 00 e3 1b 00 00 00 00 3a 3c 0f 75 89\
5a 5a 01 88 f3 55 07 e0 55 5c 00 00 00 00 00 88\
f3 55 07 88 f3 55 07 00 00 00 00 46 10 2b 00 10\
ed 55 07 d7 8a c6 76 14 ee 55 07 00 00 00 00 f1\
7e'

# Envoyer les données hexadécimales
#send_hex_data(ip_address, port_number, Hello)
# Envoyer les données hexadécimales
send_hex_data_and_receive(ip_address, port_number, Get_smith)

Connecté à 192.168.1.38 sur le port 66
Données envoyées: 7f 41 11 01 00 64 65 66 61 756c 74 2e 74 72 61 00 8f 54 18 1d 90 54 18 1d 5082 16 1d 00 00 e3 1b 00 00 00 00 fc ec 55 07 46f1 a1 77 00 00 00 00 00 00 00 00 00 00 00 00 88f3 55 07 90 54 18 1d 00 00 00 00 48 ed 55 07 1b3c 0f 75 00 00 e3 1b 00 00 00 00 3a 3c 0f 75 895a 5a 01 88 f3 55 07 e0 55 5c 00 00 00 00 00 88f3 55 07 88 f3 55 07 00 00 00 00 46 10 2b 00 10ed 55 07 d7 8a c6 76 14 ee 55 07 00 00 00 00 f17e
Paquet reçu de 1033 bytes
 20250120212020default.tra                                                                     Smith Chart VSWR)                                                               GenComm             GC747A 112200None  BDE11635  	NV3               21/1/224124703    0      ,  }^   }^   Custom                                  @KL         @Ys    HFC-12D (1/2")                          p    .                     ,            X           L      x    @      l    4      `	  	  (


Paquet reçu de 1031 bytes
	                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [18]:
def to_little_endian(paquets):
    little_endian_paquets = []
    for paquet in paquets:
        # Diviser le paquet en groupes de 2 caractères (1 octet)
        octets = [paquet[i:i+2] for i in range(0, len(paquet), 2)]
        # Inverser l'ordre des octets pour obtenir le format little-endian
        little_endian_paquet = ''.join(octets[::-1])
        little_endian_paquets.append(little_endian_paquet)
    return little_endian_paquets

#For little endian raw data frequency convertion
def to_decimal(little_endian_paquets):
    decimal_values = []
    for paquet in little_endian_paquets:
        # Convertir le paquet little-endian en entier décimal
        decimal_value = int(paquet, 16)
        decimal_values.append(decimal_value)
    return decimal_values

#For little endian raw data Real/Imag convertion
#Use only the first 20 bits
def to_decimal_20bits(little_endian_paquets):
    decimal_values = []
    for paquet in little_endian_paquets:
        # Convertir le paquet little-endian en entier décimal
        decimal_value = int(paquet[3:], 16)
        #Re(s) is negative
        if(paquet[2] == 'f'):
            decimal_value = decimal_value - 1048576
        decimal_value = decimal_value/1000000
        decimal_values.append(decimal_value)
    return decimal_values

with open(filename + '.bin', 'rb') as fichier:
    # Se positionner au 551ème octet
    fichier.seek(551)

    frequencies = []
    i = 0
    while(len(frequencies) != data_points):
        frequency = fichier.read(4)
        if len(frequency) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if(i != 120):
            frequencies.append(frequency.hex())
        else :
            fichier.read(3)
        i += 1
            
    fichier.seek(4583)
    
    real = []
    while(len(real) != data_points):
        data = fichier.read(4)
        hex_data = data.hex()
        if len(data) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if '7d5' in hex_data:
            #print("catched")
            # Retirer "7d" de hex_data
            hex_data = hex_data.replace('7d', '', 1)
            #temporary######
            #real.append("FFFFFFFF")
            ################
            # Lire un octet supplémentaire
            extra_data = fichier.read(1)
            if len(extra_data) < 1:
                break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 1 octet
            # Ajouter l'octet supplémentaire à hex_data
            hex_data += extra_data.hex()
            real.append(hex_data)
        else :
            real.append(data.hex())
    
    fichier.seek(8618)
    
    imag = []
    while(len(imag) != data_points):
        data = fichier.read(4)
        hex_data = data.hex()
        if len(data) < 4:
            break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 4 octets
        if '7d5' in hex_data:
            #print("catched")
            # Retirer "7d" de hex_data
            hex_data = hex_data.replace('7d', '', 1)
            #temporary######
            #imag.append("FFFFFFFF")
            ################
            # Lire un octet supplémentaire
            extra_data = fichier.read(1)
            if len(extra_data) < 1:
                break  # Arrêter si on atteint la fin du fichier avant d'avoir lu 1 octet
            # Ajouter l'octet supplémentaire à hex_data
            hex_data += extra_data.hex()
            imag.append(hex_data)
        else :
            imag.append(hex_data)


little_endian_paquets = to_little_endian(frequencies)
dec_frequencies = to_decimal(little_endian_paquets)
dec_frequencies = [freq / 100 for freq in dec_frequencies]
#print(dec_frequencies)

#print(real)
little_endian_paquets = to_little_endian(real)
dec_real = to_decimal_20bits(little_endian_paquets)
#print(dec_real)

#print(imag)
little_endian_paquets = to_little_endian(imag)
dec_imag = to_decimal_20bits(little_endian_paquets)
#print(little_endian_paquets)
#print(dec_imag)

Z_real, Z_imag = S_to_Z(dec_real, dec_imag)

df = pd.DataFrame({
    'frequencies': dec_frequencies,
    'real': dec_real,
    'imag' : dec_imag,
    'Zreal' : Z_real,
    'Zimag' : Z_imag
})

df.to_csv('output.csv', index=False)

In [41]:
import plotly.graph_objects as go

real = Z_real/50
imag = Z_imag.tolist()

fig = go.Figure(go.Scattersmith(imag[:5], real[:5]))
fig.show()

ValueError: The first argument to the plotly.graph_objs.Scattersmith
constructor must be a dict or
an instance of :class:`plotly.graph_objs.Scattersmith`

In [38]:
print(imag[:5])

[-385.63644816148883, -325.56611412223396, -279.3595569084273, -245.36962237679055, -219.79343342999417]


In [60]:
S = 0.9136 + 1j * (-0.2406)

# Calculer l'impédance
Z = 50 * (1 + S) / (1 - S)

# Extraire les parties réelles et imaginaires
Z_real = Z.real
Z_imag = Z.imag
print(Z.real)

0.9270730742336822
-0.3424298845630474


In [66]:
# Initialisation de la liste touchstone_lines avec les en-têtes
touchstone_lines = [
    "! Converted from CSV to Touchstone format",
    "# MHz S RI R 50"
]

# Ajout des données formatées à la liste touchstone_lines
for freq, real, imag in zip(dec_frequencies, dec_real, dec_imag):
    line = f"{freq:.6f} {real:.6f} {imag:.6f}"
    touchstone_lines.append(line)

# Écriture dans un fichier Touchstone
with open("output.s1p", "w") as file:
    for line in touchstone_lines:
        file.write(line + "\n")

print("Les données ont été écrites dans le fichier output.s1p")

Les données ont été écrites dans le fichier output.s1p
